In [1]:
from eoh import eoh
from eoh.utils.tyroParas import TyroParas
import tyro

programmatic_defaults = TyroParas(
    method="eoh",
    problem="ssscsp_all",
    llm_api_endpoint="openrouter.ai",
    llm_api_key="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",  # Use the value from the environment variable
    llm_model="google/gemini-2.0-flash-001",
    # llm_model="google/gemini-2.5-flash-preview-05-20",
    # llm_model="google/gemini-2.0-flash-lite-001",
    ec_pop_size=5,  # number of samples in each population
    ec_n_pop=5,  # number of populations
    exp_n_proc=20,  # multi-core parallel
    exp_debug_mode=False,
    # Any other parameters you want to set as a default baseline
)
params = programmatic_defaults

from eoh.problems import problems
from eoh.methods import methods

evolution = eoh.EVOL(params)

problemGenerator = problems.Probs(evolution.paras)

problem = problemGenerator.get_problem()

methodGenerator = methods.Methods(evolution.paras, problem)

method = methodGenerator.get_method()

from eoh.methods.eoh.eoh_interface_EC import InterfaceEC

interface_prob = method.prob

# interface for ec operators
interface_ec = InterfaceEC(
    method.pop_size,
    method.m,
    method.api_endpoint,
    method.api_key,
    method.llm_model,
    method.use_local_llm,
    method.llm_local_url,
    method.debug_mode,
    interface_prob,
    select=method.select,
    n_p=method.exp_n_proc,
    timeout=method.timeout,
    use_numba=method.use_numba,
)


import re
from func_timeout import func_timeout, FunctionTimedOut
from eoh.invoker import AlgorithmInvoker




DEFAULT_CLASS_NAME_HIERARCHY = ["Algorithm", "AlgorithmFIX1", "AlgorithmFIX2", "AlgorithmFIX3"]

def timed_code_evaluation(
    interface_eval, invoker: AlgorithmInvoker, timeout
):
    try:
        fitness_val, error_msg_eval = func_timeout(
            timeout,
            interface_eval.evaluate,
            args=(
                invoker,
            ),
        )
    except FunctionTimedOut:
        fitness_val = None
        error_msg_eval = f"Evaluation timed out for class '{invoker.class_to_instantiate_name}'."
    except Exception as e:
        fitness_val = None
        error_msg_eval = f"An unexpected error occurred during timed evaluation: {type(e).__name__}: {str(e)}"
    return fitness_val, error_msg_eval

def extract_python_code_from_llm_response(markdown_string):
    if not isinstance(markdown_string, str):
        return None
    match = re.search(r"```python\n(.*?)\n```", markdown_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    if "```" not in markdown_string and (
        markdown_string.strip().startswith("class ") or
        markdown_string.strip().startswith("def ") or
        markdown_string.strip().startswith("import ")
        ):
        return markdown_string.strip()
    return None

def process_code_submission_with_llm_fixing(
    initial_prompt,
    initial_code,
    base_class_code_str,
    interface_eval,
    call_llm_api,
    timeout
):
    llm_conversation_history = [
        {"role": "user", "content": initial_prompt},
        {"role": "assistant", "content": initial_code},
    ]
    successful_parent_code_definitions = []
    final_successful_code_block = None
    class_name_hierarchy = DEFAULT_CLASS_NAME_HIERARCHY 
    max_attempts = len(class_name_hierarchy)
    code_for_current_attempt = initial_code
    final_fitness_achieved = None
    for attempt_idx in range(max_attempts):
        current_class_to_instantiate = class_name_hierarchy[attempt_idx]
        definitions_for_eval = successful_parent_code_definitions + [code_for_current_attempt]

        fitness_val, error_msg_eval = timed_code_evaluation(interface_eval,AlgorithmInvoker(base_class_code_str,definitions_for_eval,current_class_to_instantiate,interface_eval.prompts.prompt_func_name), timeout)
        attempt_info = {"class_name": current_class_to_instantiate, "fitness": fitness_val}
        if fitness_val is None:
            attempt_info["error"] = error_msg_eval

        if fitness_val is not None:
            print(f"SUCCESS: Evaluation successful for {current_class_to_instantiate}! Fitness: {fitness_val}")
            final_successful_code_block = "\n".join(definitions_for_eval)
            final_fitness_achieved = fitness_val
            break
        else:
            if attempt_idx == max_attempts - 1:
                print("Max attempts reached. Could not fix the code for this submission.")
                break

            next_llm_class_name = class_name_hierarchy[attempt_idx + 1]
            parent_class_for_llm_fix = current_class_to_instantiate

            fix_prompt = f"""The Python code for class `{parent_class_for_llm_fix}` has an issue.
    When this class was used in the packing algorithm, it resulted in the following error:
    {error_msg_eval}

    Instructions:
    1. Analyze the error message and the provided code for `{parent_class_for_llm_fix}`.
    2. Create a new Python class named `{next_llm_class_name}`.
    3. This new class `{next_llm_class_name}` MUST inherit from class `{parent_class_for_llm_fix}`.
    4. In the new class (`{next_llm_class_name}`), override only the specific method(s) from `{parent_class_for_llm_fix}` that are causing the error or are directly related to fixing it. Do NOT rewrite the entire `{parent_class_for_llm_fix}` class, only provide the overridden methods or necessary additions in `{next_llm_class_name}`.
    5. The primary objective is to resolve the specified error so the code can run without this error.
    6. Ensure your fixed code is robust and adheres to the original problem's requirements if the error points to a deviation.
    7. Your response should ONLY be the Python code for the new `{next_llm_class_name}` class. Do not include any explanatory text, markdown formatting for the code block itself, or anything other than the class definition.
    """
            current_fix_request_message = {"role": "user", "content": fix_prompt}
            messages_to_send_to_llm = llm_conversation_history + [current_fix_request_message]

            llm_generated_code_fix_raw = call_llm_api(messages_to_send_to_llm)
            llm_generated_code_fix = extract_python_code_from_llm_response(llm_generated_code_fix_raw)

            if llm_generated_code_fix:
                llm_conversation_history.append(current_fix_request_message)
                llm_conversation_history.append({"role": "assistant", "content": llm_generated_code_fix_raw})
                successful_parent_code_definitions.append(code_for_current_attempt)
                code_for_current_attempt = llm_generated_code_fix
            else:
                print("LLM did not return valid code. Stopping iterative fixing for this submission.")
                break
    
    status_message = "SUCCESS" if final_successful_code_block else "FAILED"
    print(f"\n--- Processing of code submission Finished: {status_message} ---")
    return current_class_to_instantiate, final_successful_code_block, final_fitness_achieved



pop = []
operator = 'i1'
select = interface_ec.select
m = interface_ec.m
evol = interface_ec.evol
interface_eval = interface_ec.interface_eval
use_numba = interface_ec.use_numba
debug = interface_ec.debug
timeout = interface_ec.timeout

p = None
final_offspring = {
    'algorithm': None, 'code': None, 'objective': None, 'other_inf': None
}

# Inlined _standalone_get_alg function
offspring_data = {
    'algorithm': None,
    'code': None,
    'prompt': None
}
p = None


prompt_content = evol.get_prompt_i1()

--- Applying post-initialization logic ---
Successfully processed 20 instances.
--- Post-initialization complete ---
----------------------------------------- 
---              Start EoH            ---
-----------------------------------------
- output folder created -
-  parameters loaded -
- Prob local loaded 
- EoH parameters loaded -
- check LLM API
remote llm api is used ...


In [2]:
print(prompt_content)

You are to solve the **Single-Stock-Size Cutting Stock Problem (SSSCSP)** with additional constraints. You are given a list of item **types** (each with dimensions and a **quantity**) and the dimensions of a single standard container. Your goal is to pack all items into the minimum number of these identical containers.
Design a novel algorithm that, at each step, selects an available item type, **chooses one of 6 possible orientations**, and finds a valid position for it in a container.
        **Primary Objective: Minimize the total number of stock containers used.**

        Constraints:
        1. **Single Container Type:** All containers are identical. No weight constraints apply.
        2. **Complete Placement:** All items of all types must be packed.
        3. **Item Orientation:** Items can be rotated. There are 6 possible orientations.
        4. **No Overlap:** Items cannot overlap.
        5. **Boundaries:** Items must be placed fully inside the container.
        6. **Immu

In [12]:
all_code = []
for i in range(20):
    [code_all, algorithm] = evol._get_alg(prompt_content)
    all_code.append(code_all)


In [ ]:
#dump to json
import json
with open('code_2_0_flash.json', 'w') as f:
    json.dump(all_code, f, indent=4)

: 

In [ ]:
[code_all, algorithm] = evol._get_alg(prompt_content)


In [5]:
initial_code = """
import numpy as np

class Algorithm(BaseAlgorithm):
    # {This algorithm prioritizes packing larger items first and uses a greedy approach to find the first available valid placement in the least-filled compatible container, creating a new container if necessary.}
    def place_item(self, unplaced_items, trucks_in_use, truck_type):
        if not unplaced_items:
            return -1, -1, 0.0, 0.0, 0.0, 0

        container_dims = truck_type
        
        sorted_item_indices = sorted(range(len(unplaced_items)), key=lambda i: (unplaced_items[i]['length'] * unplaced_items[i]['width'] * unplaced_items[i]['height']) / (unplaced_items[i]['quantity'] if unplaced_items[i]['quantity'] > 0 else 1), reverse=True)

        for item_index in sorted_item_indices:
            item_type = unplaced_items[item_index]
            if item_type['quantity'] == 0:
                continue

            best_truck_index = -1
            best_orientation = -1
            best_pos = (0.0, 0.0, 0.0)
            best_score = float('inf')

            for truck_idx, truck in enumerate(trucks_in_use):
                if not self._check_separation_compatibility(item_type, truck):
                    continue

                placement_info = self._find_placement_in_truck(item_type, truck, container_dims)
                
                if placement_info:
                    score = self._evaluate_placement(placement_info['position'], placement_info['dimensions'], truck_idx, len(trucks_in_use))
                    if score < best_score:
                        best_score = score
                        best_truck_index = truck_idx
                        best_orientation = placement_info['orientation']
                        best_pos = placement_info['position']
            
            if best_truck_index != -1:
                return best_truck_index, item_index, best_pos[0], best_pos[1], best_pos[2], best_orientation

            if self._check_separation_compatibility(item_type, {}): # Check if an item can go into a new, empty truck
                return -1, item_index, 0.0, 0.0, 0.0, self._get_best_orientation(item_type, container_dims)

        return -1, -1, 0.0, 0.0, 0.0, 0

    def _get_best_orientation(self, item_type, container_dims):
        best_orientation = -1
        best_volume = -1
        orientations = self._get_orientations(item_type)
        for ori, dims in orientations:
            if self._is_within_container_bounds((0.0, 0.0, 0.0), dims, container_dims):
                current_volume = dims[0] * dims[1] * dims[2]
                if current_volume > best_volume:
                    best_volume = current_volume
                    best_orientation = ori
        return best_orientation if best_orientation != -1 else 0


    def _find_placement_in_truck(self, item_type, truck, container_dims):
        orientations = self._get_orientations(item_type)
        occupied_volumes = truck['occupied_volumes']
        
        for ori, dims in orientations:
            pl, pw, ph = dims
            
            # Try placing on the floor first
            possible_positions = [(0.0, 0.0, 0.0)]
            
            # Consider supporting surfaces of already placed items
            for placed_item in occupied_volumes:
                px, py, pz = placed_item['x'], placed_item['y'], placed_item['z']
                pl_other, pw_other, ph_other = placed_item['length'], placed_item['width'], placed_item['height']
                
                # Position on top of the placed item
                new_z = pz + ph_other
                if new_z + ph <= container_dims[2]:
                    # Candidate positions on the top surface of the placed item
                    # Align with the bottom-left corner of the supporting item
                    possible_positions.append((px, py, new_z))
                    # Also consider positions aligned with the corners of the supporting item's top face
                    possible_positions.append((px, py, new_z))
                    possible_positions.append((px + pl_other - pl, py, new_z))
                    possible_positions.append((px, py + pw_other - pw, new_z))
                    possible_positions.append((px + pl_other - pl, py + pw_other - pw, new_z))


            for pos in possible_positions:
                x, y, z = pos
                item_to_place_pos = (x, y, z)
                item_to_place_dims = dims
                
                if self._is_valid_placement(item_to_place_pos, item_to_place_dims, container_dims, occupied_volumes):
                    return {'position': item_to_place_pos, 'dimensions': item_to_place_dims, 'orientation': ori}
        return None

    def _evaluate_placement(self, item_pos, item_dims, truck_idx, num_trucks):
        # Heuristic to prefer placements that leave more contiguous space or pack more efficiently.
        # A simple heuristic: prioritize lower Z, then lower X, then lower Y.
        # This encourages building from the bottom up and left to right.
        x, y, z = item_pos
        l, w, h = item_dims
        
        # Penalize higher Z positions to encourage floor packing.
        z_penalty = z * 1000
        # Penalize placements that are further into the truck.
        xy_penalty = x + y * 0.1 
        
        # Further penalize using a new truck.
        truck_penalty = truck_idx if truck_idx != -1 else num_trucks * 1000
        
        return z_penalty + xy_penalty + truck_penalty"""

In [21]:
print(code_all)

import numpy as np

class Algorithm(BaseAlgorithm):
    # {The algorithm prioritizes placing items by selecting the item type with the largest volume, trying all 6 orientations for each item, and placing items in the first available position within the container, starting from the corner (0,0,0) and going along the x, y, and z axes.}
    def __init__(self, epsilon=1e-6):
        super().__init__(epsilon)
    
    def place_item(self, unplaced_items, trucks_in_use, truck_type):
        if not unplaced_items:
            return -1, -1, 0.0, 0.0, 0.0, 0
        
        item_index, item_type = self._select_item(unplaced_items)
        if item_index == -1:
            return -1, -1, 0.0, 0.0, 0.0, 0
        
        best_placement = self._find_best_placement(item_type, trucks_in_use, truck_type)
        
        if best_placement:
            truck_index, x, y, z, orientation = best_placement
            return truck_index, item_index, x, y, z, orientation
        else:
            return 

In [6]:
code_for_current_attempt = initial_code
DEFAULT_CLASS_NAME_HIERARCHY = ["Algorithm", "AlgorithmFIX1", "AlgorithmFIX2", "AlgorithmFIX3"]
class_name_hierarchy = DEFAULT_CLASS_NAME_HIERARCHY 
current_class_to_instantiate = class_name_hierarchy[0]
definitions_for_eval = [code_for_current_attempt]
interface_eval.evaluate(AlgorithmInvoker(interface_eval.base_class_code,definitions_for_eval,current_class_to_instantiate, interface_eval.prompts.prompt_func_name))



(56.55, 'Evaluation successful')